# Pipeline definition and running

This notebook serves as the core of the assignment solution for the Chicago Taxi Trips task. It defines all the necessary steps and runs the pipeline on Vertex Pipelines. The other notebooks will refer back to parts of this notebook or use the results of a pipeline run created here.

In [22]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx_v1
import tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

TensorFlow version: 2.8.1
TFX version: 1.8.0
KFP version: 1.8.12


### Setting up variables

This section sets up the GCP variables used for the pipeline.

In [232]:
GOOGLE_CLOUD_PROJECT = 'aliz-ml-spec-2022-dev'
GOOGLE_CLOUD_REGION = 'us-central1'
GCS_BUCKET_NAME = 'mormota'

PIPELINE_NAME = 'taxi-vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

PIPELINE_ROOT: gs://mormota/pipeline_root/taxi-vertex-pipelines


In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

### Prepare data

To simplify the process, we are going to use the CsvExampleGen component. To do so, we export the required portion of the dataset to a GCS bucket using the query below.

In [ ]:
%bigquery

EXPORT DATA
  OPTIONS (
    uri = 'gs://mormota/data/taxi_data/taxi_*.csv',
    format = 'CSV',
    overwrite = true,
    header = true,
    field_delimiter = ',')
AS (
  SELECT
    trip_id,
    CAST(TripStartYear AS STRING) AS TripStartYear,
     CAST(TripStartMonth AS STRING) AS TripStartMonth,
      CAST(TripStartDay AS STRING) AS TripStartDay,
       CAST(TripStartHour AS STRING) AS TripStartHour,
        CAST(TripStartMinute AS STRING) AS TripStartMinute,
         CAST(pickup_census_tract AS STRING) AS pickup_census_tract,
          CAST(dropoff_census_tract AS STRING) AS dropoff_census_tract,

    IFNULL(fare, 0) AS fare,
     IFNULL(historical_tripDuration, 0) AS historical_tripDuration,
      IFNULL(histOneWeek_tripDuration, 0) AS histOneWeek_tripDuration,
       IFNULL(historical_tripDistance, 0) AS historical_tripDistance,
        IFNULL(histOneWeek_tripDistance, 0) AS histOneWeek_tripDistance,
         IFNULL(rawDistance, 0) AS rawDistance, 
    CASE
        WHEN trip_start_timestamp >= '2021-01-01 00:00:00 UTC' THEN 'test'
        WHEN trip_start_timestamp < '2021-01-01 00:00:00 UTC' AND ABS(MOD(FARM_FINGERPRINT(trip_id), 20)) = 0 THEN 'eval'
        ELSE 'train'
    END AS selector
FROM `aliz-ml-spec-2022-dev.mormota.Demo1_MLdataset`
WHERE trip_start_timestamp >= '2020-01-01 00:00:00 UTC'
);

### Pipeline modules

The following blocks create the preprocessing and model module code and upload the files to GCS. This last step is necessary for TFX to be able to use our custom code in the different components.

#### Preprocessing

The preprocessing transforms the categorical features into vocabularies and standardises the numerical features.

#### Model

The model is a simple DNN that is made up of two blocks. The first processes the inputs and embeds taxi trips in an implicit feature space. The second is a dense block that leads to the final predictions.

#### Hyperparameter tuning

The hyperparameter tuning section uses Keras tuning to perform a hyperparameter search on the previously defined model.

In [243]:
preprocess_module_file = 'preprocess.py'

In [244]:
%%writefile {preprocess_module_file}

import tensorflow_transform as transform
import tensorflow as tf

_VOCAB_FEATURE_KEYS = [
    'TripStartYear',
    'TripStartMonth',
    'TripStartDay',
    'TripStartHour',
    'TripStartMinute',
    'pickup_census_tract',
    'dropoff_census_tract'
]

_VOCAB_SIZE = 10
_OOV_SIZE = 5

_FARE_KEY = 'fare'

_LABEL_KEY = 'fare'

_DENSE_FLOAT_FEATURE_KEYS = [
    'fare',
 'historical_tripDuration',
 'histOneWeek_tripDuration',
 'historical_tripDistance',
 'histOneWeek_tripDistance',
 'rawDistance',
]

_BUCKET_FEATURE_KEYS = []

_FEATURE_BUCKET_COUNT = 10

_CATEGORICAL_FEATURE_KEYS = []

def _transformed_name(key):
  return key# + '_xf'

def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = transform.scale_to_z_score(inputs[key])

  for key in _VOCAB_FEATURE_KEYS:
    outputs[_transformed_name(
        key)] = transform.compute_and_apply_vocabulary(
            inputs[key],
            top_k=_VOCAB_SIZE,
            num_oov_buckets=_OOV_SIZE)

  for key in _BUCKET_FEATURE_KEYS:
    outputs[_transformed_name(key)] = transform.bucketize(
        inputs[key], _FEATURE_BUCKET_COUNT)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = inputs[key]

  taxi_fare = inputs[_FARE_KEY]
  tips = inputs[_LABEL_KEY]
  outputs[_transformed_name(_LABEL_KEY)] = tf.where(
      tf.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

Overwriting preprocess.py


In [245]:
_trainer_module_file = 'taxi_trainer.py'

In [246]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_data_validation as tfdv
from tfx import v1 as tfx_v1
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [#'trip_id',
 'TripStartYear',
 'TripStartMonth',
 'TripStartDay',
 'TripStartHour',
 'TripStartMinute',
 'pickup_census_tract',
 'dropoff_census_tract',
 #'fare',
 'historical_tripDuration',
 'histOneWeek_tripDuration',
 'historical_tripDistance',
 'histOneWeek_tripDistance',
 'rawDistance',
 'selector']
_LABEL_KEY = 'fare'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


def _get_hyperparameters() -> keras_tuner.HyperParameters:
  """Returns hyperparameters for building Keras model."""
  hp = keras_tuner.HyperParameters()
    
  hp.Choice('learning_rate', [1e-2, 1e-3], default=1e-2)
  hp.Int('num_layers', 1, 3, default=2)
  return hp


def _input_fn(file_pattern: List[str],
              data_accessor: tfx_v1.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model(hparams: keras_tuner.HyperParameters) -> tf.keras.Model:
  """Creates a DNN Keras model for predicting taxi trips fare for the Chicago Taxi Trips assignment.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(int(hparams.get('num_layers'))):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(hparams.get('learning_rate')),
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[keras.metrics.MeanSquaredError()])

  model.summary(print_fn=logging.info)
  return model

def tuner_fn(fn_args: tfx.components.FnArgs) -> tfx.components.TunerFnResult:
  """Build the tuner using the KerasTuner API.
  Args:
    fn_args: Holds args as name/value pairs.
      - working_dir: working dir for tuning.
      - train_files: List of file paths containing training tf.Example data.
      - eval_files: List of file paths containing eval tf.Example data.
      - train_steps: number of train steps.
      - eval_steps: number of eval steps.
      - schema_path: optional schema of the input data.
      - transform_graph_path: optional transform graph produced by TFT.
  Returns:
    A namedtuple contains the following:
      - tuner: A BaseTuner that will be used for tuning.
      - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
  """
  tuner = keras_tuner.RandomSearch(
      _make_keras_model,
      max_trials=6,
      hyperparameters=_get_hyperparameters(),
      allow_new_entries=False,
      objective=keras_tuner.Objective('val_root_mean_squared_error', 'max'),
      directory=fn_args.working_dir,
      project_name='taxi_tuning')

  transform_graph = tft.TFTransformOutput(fn_args.transform_graph_path)

  train_dataset = base.input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      transform_graph,
      base.TRAIN_BATCH_SIZE)

  eval_dataset = base.input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      transform_graph,
      base.EVAL_BATCH_SIZE)

  return tfx.components.TunerFnResult(
      tuner=tuner,
      fit_kwargs={
          'x': train_dataset,
          'validation_data': eval_dataset,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      })


def run_fn(fn_args: tfx_v1.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  schema = tfdv.load_schema_text(fn_args.schema_path)
  #schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)
  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting taxi_trainer.py


Copy the module file to GCS which can be accessed from the pipeline components.
Because model training happens on GCP, we need to upload this model definition. 

Otherwise, you might want to build a container image including the module file
and use the image to run the pipeline.

In [247]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

Copying file://taxi_trainer.py [Content-Type=text/x-python]...
/ [1 files][  5.5 KiB/  5.5 KiB]                                                
Operation completed over 1 objects/5.5 KiB.                                      


In [248]:
!gsutil cp {preprocess_module_file} {MODULE_ROOT}/

Copying file://preprocess.py [Content-Type=text/x-python]...
/ [1 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 1 objects/1.8 KiB.                                      


### Pipeline definition

The following block defines our TFX pipeline. It is made up of the following components:

- CsvExampleGen: Loads our dataset from Google Cloud Storage
- SchemaGen: Creates a TF schema from our dataset to be used in later components
- ExampleValidator: Allows us to check for data skew later on
- Transform: Performs preprocessing on the data
- Tuner: Performs hyperparameter tuning
- Trainer: Trains the model
- Pusher: Deploys the model to the endpoint on Vertex AI

In [249]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str, project_id: str,
                     endpoint_name: str, region: str,
                     ) -> tfx_v1.dsl.Pipeline:
  """Defines a pipeline for the Chicago Taxi Trips assignment"""

  example_gen = tfx_v1.components.CsvExampleGen(input_base='gs://mormota/data/taxi_data')
  compute_eval_stats = tfx_v1.components.StatisticsGen(
      examples=example_gen.outputs['examples'],
      )
  schema_gen = tfx_v1.components.SchemaGen(
    statistics=compute_eval_stats.outputs['statistics'])
    
  validate_stats = tfx_v1.components.ExampleValidator(
      statistics=compute_eval_stats.outputs['statistics'],
      schema=schema_gen.outputs['schema']
      )

  transform = tfx_v1.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=preprocess_module_file)
    
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }


  trainer = tfx_v1.components.Trainer(
      module_file=module_file,
      #examples=example_gen.outputs['examples'],
      examples=transform.outputs['transformed_examples'],
      transform_graph=transform.outputs['transform_graph'],
      schema=schema_gen.outputs['schema'],
      train_args=tfx_v1.proto.TrainArgs(num_steps=100),
      eval_args=tfx_v1.proto.EvalArgs(num_steps=5))

  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      'machine_type': 'n1-standard-4',
  }

  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  pusher = tfx_v1.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx_v1.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx_v1.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx_v1.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx_v1.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      compute_eval_stats,
      schema_gen,
      validate_stats,
      transform,
      trainer,
      pusher,
  ]

  return tfx_v1.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

### Running the pipeline on Vertex Pipelines.

The following section compiles the pipeline using the Kubeflow V2 SDK and then starts a pipeline run on Vertex Pipelines

In [250]:
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx_v1.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx_v1.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        serving_model_dir=SERVING_MODEL_DIR))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/jupyter/new_dir/preprocess.py' (including modules: ['kubeflow_v2_runner', 'local_runner', 'preprocess', 'penguin_trainer', 'kubeflow_runner', 'taxi_trainer']).
INFO:absl:User module package has hash fingerprint version f11ac71028092cf23345c26b66cebd05a08a0516d14f82a979e4f3c34a01fc2c.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpq0ah4pez/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp9u6mpuxq', '--dist-dir', '/tmp/tmpjh5nf92s']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying kubeflow_v2_runner.py -> build/lib
copying local_runner.py -> build/lib
copying preprocess.py -> build/lib
copying penguin_trainer.py -> build/lib
copying kubeflow_runner.py -> build/lib
copying taxi_trainer.py -> build/lib
installing to /tmp/tmp9u6mpuxq
running install
running install_lib
copying build/lib/taxi_trainer.py -> /tmp/tmp9u6mpuxq
copying build/lib/penguin_trainer.py -> /tmp/tmp9u6mpuxq
copying build/lib/kubeflow_v2_runner.py -> /tmp/tmp9u6mpuxq
copying build/lib/local_runner.py -> /tmp/tmp9u6mpuxq
copying build/lib/preprocess.py -> /tmp/tmp9u6mpuxq
copying build/lib/kubeflow_runner.py -> /tmp/tmp9u6mpuxq
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_le

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at 'gs://mormota/pipeline_root/taxi-vertex-pipelines/_wheels/tfx_user_code_Transform-0.0+f11ac71028092cf23345c26b66cebd05a08a0516d14f82a979e4f3c34a01fc2c-py3-none-any.whl'; target user module is 'preprocess'.
INFO:absl:Full user module path is 'preprocess@gs://mormota/pipeline_root/taxi-vertex-pipelines/_wheels/tfx_user_code_Transform-0.0+f11ac71028092cf23345c26b66cebd05a08a0516d14f82a979e4f3c34a01fc2c-py3-none-any.whl'
INFO:absl:Generating ephemeral wheel package for '/tmp/tmp92zglyww/taxi_trainer.py' (including modules: ['taxi_trainer']).
INFO:absl:User module package has hash fingerprint version 86a8dae30f9470ac452bc47323cba22c89aa73987d7e6157d4dc3d89616a1103.
INFO:absl:Executing: ['/opt/conda/bin/pyt

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying taxi_trainer.py -> build/lib
installing to /tmp/tmpb7i255j9
running install
running install_lib
copying build/lib/taxi_trainer.py -> /tmp/tmpb7i255j9
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpb7i255j9/tfx_user_code_Trainer-0.0+86a8dae30f9470ac452bc47323cba22c89aa73987d7e6157d4dc3d89616a1103-py3.7.egg-info
running install_scripts
creating /tmp/tmpb7i255j9/tfx_user_code_Trainer-0.0+86a8dae30f9470ac452bc47323cba22c8

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
INFO:absl:Successfully built user code wheel distribution at 'gs://mormota/pipeline_root/taxi-vertex-pipelines/_wheels/tfx_user_code_Trainer-0.0+86a8dae30f9470ac452bc47323cba22c89aa73987d7e6157d4dc3d89616a1103-py3-none-any.whl'; target user module is 'taxi_trainer'.
INFO:absl:Full user module path is 'taxi_trainer@gs://mormota/pipeline_root/taxi-vertex-pipelines/_wheels/tfx_user_code_Trainer-0.0+86a8dae30f9470ac452bc47323cba22c89aa73987d7e6157d4dc3d89616a1103-py3-none-any.whl'


In [251]:
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/639006805448/locations/us-central1/pipelineJobs/taxi-vertex-pipelines-20220608053153


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/639006805448/locations/us-central1/pipelineJobs/taxi-vertex-pipelines-20220608053153


To use this PipelineJob in another session:


INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/639006805448/locations/us-central1/pipelineJobs/taxi-vertex-pipelines-20220608053153')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/639006805448/locations/us-central1/pipelineJobs/taxi-vertex-pipelines-20220608053153')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/taxi-vertex-pipelines-20220608053153?project=639006805448


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/taxi-vertex-pipelines-20220608053153?project=639006805448
